<a href="https://colab.research.google.com/github/201524495/201524495/blob/main/incontext_learning_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Zero/Few-shot In-context Learning 실습 노트북
**작성일**: 2025-07-18 by Yeongtak Oh (Credit : DSAIL LAb, SNU)

**목표**:  
- Zero-shot 및 Few-shot 학습 방식 이해  
- Chain-of-Thought 및 Tool-use 프롬프트 실습  
- 사용자 입력 기반 인터랙션 실습  
- 결과 CSV 저장, 체크리스트, 퀴즈 포함

---
> 본 노트북은 HD현대 실습을 위해 교육용 자료로서 준비되었으며, PyTorch와 HuggingFace Transformers 라이브러리를 사용합니다.


In [ ]:
!pip install transformers datasets ipywidgets --quiet


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

 # https://huggingface.co/gpt2-medium
model_name = "gpt2-medium" # "Qwen/Qwen3-0.6B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
model.eval()

log = []


In [ ]:
def generate_completion(prompt, max_new_tokens=50):
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
    with torch.no_grad():
        output_ids = model.generate(
            input_ids,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=0.7,
            top_p=0.95,
            pad_token_id=tokenizer.eos_token_id
        )
    output = tokenizer.decode(output_ids[0][input_ids.shape[-1]:], skip_special_tokens=True)
    log.append({"prompt": prompt, "response": output})
    return output



## 실습 체크리스트

- 1) Zero-shot 번역 실습 실행  
- 2) Few-shot 번역 실습 실행  
- 3) 감정 분석 예제 실행  
- 4) 사용자 직접 프롬프트 실습  
- 5) Chain-of-Thought 예제 실행  
- 6) Tool-use 예제 실행  
- 결과를 CSV로 저장


In [ ]:
# zero-shot은 demonstration x
print("Zero-shot 예제")
prompt = "Translate English to French: I like pizza."
print("Prompt:", prompt)
print("Generated:", generate_completion(prompt))

# few-shot은 demonstration o
print("\n Few-shot 예제")
few_shot = (
    "Translate English to French:\n"
    "English: Hello.\nFrench: Bonjour.\n"
    "English: Good night.\nFrench: Bonne nuit.\n"
    "English: I like pizza.\nFrench:"
)
print("Prompt:\n", few_shot)
print("Generated:", generate_completion(few_shot))


In [ ]:
# example 1: sentimental analysis
print("\n 감정 분석 Few-shot")
emotion_prompt = (
    "Tweet: I hate everything.\nSentiment: Negative\n"
    "Tweet: I love sunshine.\nSentiment: Positive\n"
    "Tweet: I want to cry.\nSentiment:"
)
print(generate_completion(emotion_prompt))

# CoT example
print("\n Chain-of-Thought 예제")
cot = (
    "Q: If a train has 5 cars and each car has 20 people, how many total people?\n"
    "A: Let's think step by step. Each car has 20 people. There are 5 cars. So 5 * 20 = 100. Answer: 100"
    "\n\nQ: There are 3 boxes. Each box has 4 apples. How many apples?\nA:"
)
print(generate_completion(cot))

# Tool use example
print("\n Tool-use 예제")
tool_prompt = (
    "Question: What is 15 * 12?\n"
    "Let's use a calculator: 15 * 12 = 180.\nAnswer: 180\n"
    "Question: What is 9 * 11?\nLet's use a calculator:"
)
print(generate_completion(tool_prompt))


In [ ]:
# With on your own prompts (PLAYGROUND)
prompt_input = widgets.Textarea(
    value='Input your prompt here...',
    placeholder='Enter any instruction or query',
    description='Prompt:',
    layout=widgets.Layout(width='100%', height='100px')
)
button = widgets.Button(description='Generate')
output = widgets.Output()

@output.capture()
def on_click(b):
    output.clear_output()
    prompt = prompt_input.value
    result = generate_completion(prompt)
    print("Generated:", result)

button.on_click(on_click)
display(prompt_input, button, output)


In [ ]:
# Output save files
df = pd.DataFrame(log)
df.to_csv("incontext_outputs.csv", index=False)
print("💾 Saved as incontext_outputs.csv")



## 📝 퀴즈 (5분)

1. Zero-shot learning과 Few-shot learning의 차이를 서술하시오.  
2. Chain-of-Thought prompting이 모델 성능 향상에 어떤 영향을 미치는가?  
3. Tool-use prompting의 한계점은 무엇인가?

> 답안은 아래 셀에 작성하세요.


In [ ]:
# 여기에 작성하세요.

### 답변  

#### 1. **Zero-shot learning vs. Few-shot learning**  
- **Zero-shot learning**:  
  - 모델이 **학습 시 특정 태스크나 데이터를 경험하지 않았음에도**, **추론 시 단일 예시 없이** 새로운 태스크를 수행하는 방식입니다.  
  - 예: "이 문장은 긍정적입니까? 부정적입니까?"와 같이 **명시적 예시 없이** 분류를 요청.  
  - **장점**: 데이터 부족 환경에서 유연성 제공.  
  - **단점**: 복잡한 태스크에서는 성능 저하 가능성.  

- **Few-shot learning**:  
  - **1~몇 개의 예시**를 제공하여 모델이 패턴을 학습하게 하는 방식입니다.  
  - 예: "긍정: '좋은 제품입니다!', 부정: '불만족스러웠습니다.' → '이 서비스는 훌륭합니다!'는 어떤가요?"  
  - **장점**: 예시를 통해 성능 향상 가능.  
  - **단점**: 예시의 질과 수량에 따라 결과가 변동.  

**핵심 차이**: Zero-shot은 **예시 없이**, Few-shot은 **최소한의 예시로** 태스크를 수행합니다.  

---

#### 2. **Chain-of-Thought Prompting의 영향**  
- **원리**: 모델에게 **중간 단계의 추론 과정을 명시적으로 설명**하도록 유도합니다.  
  - 예: "이 문제를 푸는 방법을 단계별로 설명해 주세요."  
- **효과**:  
  - **복잡한 태스크**(수학 문제, 논리 추론)에서 정확도 향상.  
  - 모델이 **내재된 논리적 구조를 명시적으로 활용**하게 됨.  
  - **예측 가능성과 해석성** 증가.  
- **제한**:  
  - 단순 태스크에서는 과도한 복잡성 유발 가능성.  
  - 추론 단계가 잘못 생성될 경우 최종 결과 오류 발생.  

---

#### 3. **Tool-use Prompting의 한계점**  
- **정의**: 외부 도구(예: 계산기, API)를 활용해 모델의 답변을 보완하는 방식.  
- **한계**:  
  1. **도구 의존성**:  
     - 도구의 **정확성, 업데이트 상태**에 따라 결과가 변동.  
     - 예: 오래된 데이터베이스를 사용하면 실시간 정보 제공 불가.  
  2. **통합 복잡성**:  
     - 도구와의 **인터페이스 설계**, 오류 처리 등 추가 개발 필요.  
  3. **추론 오류**:  
     - 모델이 **도구 사용 시점이나 방식을 잘못 판단**할 수 있음.  
     - 예: 불필요한 도구 호출로 인한 비효율.  
  4. **보안 및 비용**:  
     - 외부 API 사용 시 **비용 증가** 또는 **데이터 유출 리스크** 발생.  

---

### 요약  
| 항목 | Zero-shot vs. Few-shot | Chain-of-Thought | Tool-use Prompting |  
|------|------------------------|------------------|--------------------|  
| **핵심** | 예시 유무에 따른 태스크 수행 | 추론 단계 명시 | 외부 도구 활용 |  
| **장점** | 데이터 독립성, 유연성 | 복잡 태스크 성능 향상 | 정확도 보완 |  
| **단점** | 성능 불안정, 예시 의존 | 과도한 복잡성 | 도구 의존성, 비용 증가 |  

